<a href="https://colab.research.google.com/github/ykitaguchi77/grav_bootcamp/blob/master/YOLOv5_dataset_to_bootcamp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5用データセット --> Bootcamp用セット作成**

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd
import glob
import os


for dataset in ["train", "valid"]:
    dataset_path = f"/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/{dataset}" #train or valid 

    txt_files = []  # txtファイルのパスを格納するリスト
    image_paths, name_list, label_list = [],  [], []

    # labelフォルダ内のテキストの中の１番目の数字がラベル
    for file_name in os.listdir(f"{dataset_path}/labels"):
        if file_name.endswith('.txt'):
            file_path = f"{dataset_path}/labels/{file_name}"
            name = os.path.basename(file_path).split(".")[0] 
            name_list.append(name.split("-")[0]) #患者IDを抜き出す
            image_paths.append(f"{dataset_path}/images/{name}.JPG")

            with open(file_path, 'r') as f:
                for line in f:
                    label = line.split()[0]
                    label_list.append(label)

    # データフレームを作成する
    df = pd.DataFrame({'image_path': image_paths, 'name': name_list, 'label': label_list})
    print(f"{dataset}: {len(df)}")

    # データフレームを表示する
    df.to_csv(f'/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/{dataset}_list.csv', index=False)




train: 2649
valid: 664


#**Valid_dataset --> hum_evalへ**

validデータセットから、なるべく患者が被らないように66×3セットの人力評価用データセットを作成する

In [71]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Deep_learning/GO_extended_dataset/periocular_for_YOLO_training/valid_list.csv')
# df = df.sort_values('name') #nameで並べ替え
# df = df.sort_values('label') #labelで並べ替え
# df = df.reset_index() #index振り直し
# df = df.drop('index', axis=1) #indexの列を削除
# df
df

,image_path,name,label
0,/content/drive/MyDrive/Deep_learning/GO_extend...,6055,0
1,/content/drive/MyDrive/Deep_learning/GO_extend...,5905,0
2,/content/drive/MyDrive/Deep_learning/GO_extend...,5047,0
3,/content/drive/MyDrive/Deep_learning/GO_extend...,4219,0
4,/content/drive/MyDrive/Deep_learning/GO_extend...,5807,0
...,...,...,...
659,/content/drive/MyDrive/Deep_learning/GO_extend...,6298,1
660,/content/drive/MyDrive/Deep_learning/GO_extend...,6599,1
661,/content/drive/MyDrive/Deep_learning/GO_extend...,4813,1
662,/content/drive/MyDrive/Deep_learning/GO_extend...,3484,1


In [73]:
import pandas as pd

def extract_dataset(df):
    # nameが重複していないdfを作る（かぶっているものをランダムにdropした）
    rows_no_name_duplicate = df.groupby("name").apply(lambda x: x.sample(1)).reset_index(drop=True)

    # "label"が0の行を取得
    label_0_rows = rows_no_name_duplicate[rows_no_name_duplicate["label"]==0]

    # "label"が1の行を取得
    label_1_rows = rows_no_name_duplicate[rows_no_name_duplicate["label"]==1]

    # ランダムに33個選択
    random_label_0_rows = label_0_rows.sample(n=33, random_state=1)

    # ランダムに33個選択
    random_label_1_rows = label_1_rows.sample(n=33, random_state=1)

    selected_index = pd.concat([random_label_0_rows, random_label_1_rows]).index.tolist()
    df_remain = df.drop(selected_index)

    return random_label_0_rows, random_label_1_rows, df_remain




In [74]:
step0_cont, step0_grav, df = extract_dataset(df)
step1_cont, step1_grav, df = extract_dataset(df)
step2_cont, step2_grav, df = extract_dataset(df)

KeyError: ignored

In [69]:
len(df_remain)

598